# Read Diamond_HS_Aug

In [1]:
!pip install awswrangler

In [2]:
import awswrangler as wr

import boto3 

import pandas as pd
import numpy as np
import datetime as dt
import os              
#(interacting with several os system)

import gc 

import pandas as pd
import numpy as np
import datetime as dt
import os

import fsspec               
#(provide a unified pythonic interface to local, remote and embedded file systems)

import s3fs                 
#( Pythonic file interface to S3)

from string import Template 
#(template usuing $ based substitution)

import io

import argparse             
#(to add custom arguments eg print("Hello" + unser input arg (args.name))

import warnings

import subprocess
import asyncio, sys

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', None)

In [4]:
Health_State_2023= pd.read_csv("s3://intl-euro-uk-datascientist-prod/Abhiya/HEALTH_STATE_2023/8_HEALTH_STATE_CCSR_2023_AUG.csv",
                               usecols=['icd_10_cm_code','member_id',
                                        'claim_location_country_nm','region',
                                        'unique_member_id_key', 'local_currency_net_amt'
                                        ])


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [5]:
Health_State_2023 = Health_State_2023.rename(columns={
                                                      'icd_10_cm_code':'HS_icd_10_cm_code',
                                                      'member_id':'HS_member_id',
                                                      'claim_location_country_nm':'HS_claim_location_country_nm',
                                                      'region':'HS_region',
                                                      'unique_member_id_key':'HS_unique_member_id_key',
                                                      'local_currency_net_amt':'HS_local_currency_net_amt'
                                                     })

In [6]:
Health_State_2023.shape

(572373, 6)

# Read CCSR_2023

In [7]:
CCSR_CCS=pd.read_csv("s3://intl-euro-uk-datascientist-prod/Abhiya/CCSR_CCS_23_19_comparison_FULL.csv",
                usecols=['ICD_code_2023','ICD_10_CM_CODE_DESCRIPTION_2023',
                'Default_CCSR_CATEGORY_IP_2023','Default_CCSR_CATEGORY_DESCRIPTION_IP_2023',
                'Default_CCSR_CATEGORY_OP_2023','Default_CCSR_CATEGORY_DESCRIPTION_OP_2023',
                'CCSR_CATEGORY_1_2023','CCSR_CATEGORY_1_DESCRIPTION_2023',
                'CODE_range_first_three_characters',
                 'CCSR_Body_System_Abbreviation_2023','ICD_10_code_book_chapter_2023',
                'icd_code_2019','ICD_10_CM_CODE_DESCRIPTION_2019',
                'CCS_CATEGORY_2019','CCS_CATEGORY_DESCRIPTION_2019',
                'CCS_LVL_1_code_2019','CCS_LVL_1_Description_2019',
                'CCS_LVL_2_code_2019','CCS_LVL_2_Description_2019']
                            
                )

 

#CCS_chapter_number_2019',
#CCS_Body_System_Abbreviation_2019',
#IVD_9_code_diagnosis chapters'

/tmp/ipykernel_6310/3052546784.py:1: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  CCSR_CCS=pd.read_csv("s3://intl-euro-uk-datascientist-prod/Abhiya/CCSR_CCS_23_19_comparison_FULL.csv",


In [8]:
CCSR_CCS=CCSR_CCS.rename(columns={'CODE_range_first_three_characters':'ICD_2023_code_abr'})

In [9]:
Health_State_2023_map_CCSR_CCS=pd.merge(left=Health_State_2023,right=CCSR_CCS, how='left',\
                        left_on='HS_icd_10_cm_code',right_on='ICD_code_2023', indicator='ICD_CODE_BUCKET_HS_CCSR_2023') 


In [10]:
Health_State_2023_map_CCSR_CCS['ICD_CODE_BUCKET_HS_CCSR_2023'].value_counts(normalize=True)\
.mul(100).round(2).astype(str)+'%'

ICD_CODE_BUCKET_HS_CCSR_2023
both          91.34%
left_only      8.66%
right_only      0.0%
Name: proportion, dtype: object

# Read CCS_2019

In [11]:
ICD_9_CHAPTER_NAME= pd.read_csv("s3://intl-euro-uk-datascientist-prod/Abhiya/icd_files_2019-2023/Chapter_CCS_icd_9.csv")

In [12]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER= pd.merge (left=Health_State_2023_map_CCSR_CCS, right=ICD_9_CHAPTER_NAME, how='left',\
                               left_on='CCS_LVL_1_code_2019', right_on='CCS_chapter_number_2019',\
                                indicator='CCS_LEVEL1_CHAPTER_NAME_BUCKET')

In [13]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER.shape

(572373, 30)

In [14]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER['CCS_LEVEL1_CHAPTER_NAME_BUCKET'].value_counts(normalize=True)\
.mul(100).round(2).astype(str)+'%' 

CCS_LEVEL1_CHAPTER_NAME_BUCKET
both          88.59%
left_only     11.41%
right_only      0.0%
Name: proportion, dtype: object

# CCSR_2023

In [15]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER['HS_local_currency_net_amt'].sum()

69505738.86000001

In [16]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER.groupby('CCSR_Body_System_Abbreviation_2023')\
.agg({'HS_unique_member_id_key': 'count', 'HS_local_currency_net_amt':'sum'})

,HS_unique_member_id_key,HS_local_currency_net_amt
CCSR_Body_System_Abbreviation_2023,,
BLD,4718,1250152.59
CIR,13347,4225774.41
DIG,12495,2626491.04
EAR,3577,351681.54
END,22356,1745146.07
EXT,291,34918.73
EYE,8832,862213.42
FAC,228272,21078325.97
GEN,19745,2846172.36


In [17]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER[('CCSR_Body_System_Abbreviation_2023')].isnull().sum()

49558

# CCS_2019

In [18]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER.groupby('CCS_Body_System_Abbreviation_2019')\
.agg({'HS_unique_member_id_key': 'count', 'HS_local_currency_net_amt':'sum'})

,HS_unique_member_id_key,HS_local_currency_net_amt
CCS_Body_System_Abbreviation_2019,,
BLD,4319,1174648.18
CIR,19128,5011570.31
DIG,15909,2744238.92
END,25116,1940234.40
GEN,21902,2978736.90
INF,27916,2647114.58
INJ,13540,2513616.10
MAL,1556,847816.15
MBD,22111,2657945.91


In [19]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER[('CCS_Body_System_Abbreviation_2019')].isnull().sum()

65310

## Maping ICD10 in diamond HS subset to 2023 CCSR chapters

In [20]:
ICD_CHAPTER_2023= pd.read_csv("s3://intl-euro-uk-datascientist-prod/Abhiya/icd_files_2019-2023/Code_chapter_2023.csv")

In [21]:
ICD_CHAPTER_2023=ICD_CHAPTER_2023.rename(columns={'CCSR_Body_System_Abbreviation_2023':'CCSR_Body_sys_ABR_mapped_HS_CODE',
                                                 'ICD_10_code_book_chapter_2023':'ICD_10_chapter_mapped_HS_CODE'})

In [22]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER['HS_ICD_code_Abr']=HEALTH_STATE_MAP_CCSR_CCS_CHAPTER['HS_icd_10_cm_code'].str[0:3]

In [23]:
HEALTH_STATE_MAP_final= pd.merge (left=HEALTH_STATE_MAP_CCSR_CCS_CHAPTER, right=ICD_CHAPTER_2023, how='left',\
                               left_on='HS_ICD_code_Abr', right_on='CODE_range_first_three_characters',\
                                indicator='CCSR_CHAPTER_CODE_BUCKET')

In [24]:
HEALTH_STATE_MAP_final['CCSR_CHAPTER_CODE_BUCKET'].value_counts(normalize=True)\
.mul(100).round(2).astype(str)+'%' 

CCSR_CHAPTER_CODE_BUCKET
both          99.75%
left_only      0.25%
right_only      0.0%
Name: proportion, dtype: object

# Extra or DUPLICATE ROWS 

In [25]:
HEALTH_STATE_MAP_final.shape

(572373, 35)

In [26]:
HEALTH_STATE_MAP_final.head(3)

,HS_unique_member_id_key,HS_claim_location_country_nm,HS_region,HS_icd_10_cm_code,HS_local_currency_net_amt,HS_member_id,ICD_code_2023,ICD_10_CM_CODE_DESCRIPTION_2023,Default_CCSR_CATEGORY_IP_2023,Default_CCSR_CATEGORY_DESCRIPTION_IP_2023,Default_CCSR_CATEGORY_OP_2023,Default_CCSR_CATEGORY_DESCRIPTION_OP_2023,CCS_CATEGORY_2019,CCS_CATEGORY_DESCRIPTION_2019,icd_code_2019,ICD_10_CM_CODE_DESCRIPTION_2019,CCS_LVL_1_code_2019,CCSR_CATEGORY_1_2023,CCSR_CATEGORY_1_DESCRIPTION_2023,CCS_LVL_1_Description_2019,CCS_LVL_2_code_2019,CCS_LVL_2_Description_2019,CCSR_Body_System_Abbreviation_2023,ICD_2023_code_abr,ICD_10_code_book_chapter_2023,ICD_CODE_BUCKET_HS_CCSR_2023,CCS_chapter_number_2019,CCS_Body_System_Abbreviation_2019,IVD_9_code_diagnosis chapters,CCS_LEVEL1_CHAPTER_NAME_BUCKET,HS_ICD_code_Abr,CODE_range_first_three_characters,CCSR_Body_sys_ABR_mapped_HS_CODE,ICD_10_chapter_mapped_HS_CODE,CCSR_CHAPTER_CODE_BUCKET
0,11598608,United States of America (the),Americas,I2694,54.98,2.101605e+10,I2694,Multiple subsegmental pulmonary emboli without...,CIR013,Acute pulmonary embolism,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,NaN,NaN,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,CIR,I26,Diseases of the circulatory system,both,NaN,NaN,NaN,left_only,I26,I26,CIR,Diseases of the circulatory system,both
1,11598608,United States of America (the),Americas,I2694,84.28,2.101605e+10,I2694,Multiple subsegmental pulmonary emboli without...,CIR013,Acute pulmonary embolism,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,NaN,NaN,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,CIR,I26,Diseases of the circulatory system,both,NaN,NaN,NaN,left_only,I26,I26,CIR,Diseases of the circulatory system,both
2,11598608,United States of America (the),Americas,I2694,248.85,2.101605e+10,I2694,Multiple subsegmental pulmonary emboli without...,CIR013,Acute pulmonary embolism,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,NaN,NaN,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,CIR,I26,Diseases of the circulatory system,both,NaN,NaN,NaN,left_only,I26,I26,CIR,Diseases of the circulatory system,both


## CHECK FOR DUPLICATES-here 50% data is duplicated

In [27]:
dup=HEALTH_STATE_MAP_final[HEALTH_STATE_MAP_final.duplicated(['HS_icd_10_cm_code','HS_unique_member_id_key'])]

In [28]:
dup.head(3)


,HS_unique_member_id_key,HS_claim_location_country_nm,HS_region,HS_icd_10_cm_code,HS_local_currency_net_amt,HS_member_id,ICD_code_2023,ICD_10_CM_CODE_DESCRIPTION_2023,Default_CCSR_CATEGORY_IP_2023,Default_CCSR_CATEGORY_DESCRIPTION_IP_2023,Default_CCSR_CATEGORY_OP_2023,Default_CCSR_CATEGORY_DESCRIPTION_OP_2023,CCS_CATEGORY_2019,CCS_CATEGORY_DESCRIPTION_2019,icd_code_2019,ICD_10_CM_CODE_DESCRIPTION_2019,CCS_LVL_1_code_2019,CCSR_CATEGORY_1_2023,CCSR_CATEGORY_1_DESCRIPTION_2023,CCS_LVL_1_Description_2019,CCS_LVL_2_code_2019,CCS_LVL_2_Description_2019,CCSR_Body_System_Abbreviation_2023,ICD_2023_code_abr,ICD_10_code_book_chapter_2023,ICD_CODE_BUCKET_HS_CCSR_2023,CCS_chapter_number_2019,CCS_Body_System_Abbreviation_2019,IVD_9_code_diagnosis chapters,CCS_LEVEL1_CHAPTER_NAME_BUCKET,HS_ICD_code_Abr,CODE_range_first_three_characters,CCSR_Body_sys_ABR_mapped_HS_CODE,ICD_10_chapter_mapped_HS_CODE,CCSR_CHAPTER_CODE_BUCKET
1,11598608,United States of America (the),Americas,I2694,84.28,2.101605e+10,I2694,Multiple subsegmental pulmonary emboli without...,CIR013,Acute pulmonary embolism,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,NaN,NaN,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,CIR,I26,Diseases of the circulatory system,both,NaN,NaN,NaN,left_only,I26,I26,CIR,Diseases of the circulatory system,both
2,11598608,United States of America (the),Americas,I2694,248.85,2.101605e+10,I2694,Multiple subsegmental pulmonary emboli without...,CIR013,Acute pulmonary embolism,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,NaN,NaN,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,CIR,I26,Diseases of the circulatory system,both,NaN,NaN,NaN,left_only,I26,I26,CIR,Diseases of the circulatory system,both
3,11598608,United States of America (the),Americas,I2694,33.50,2.101605e+10,I2694,Multiple subsegmental pulmonary emboli without...,CIR013,Acute pulmonary embolism,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,NaN,NaN,CIR013,Acute pulmonary embolism,NaN,NaN,NaN,CIR,I26,Diseases of the circulatory system,both,NaN,NaN,NaN,left_only,I26,I26,CIR,Diseases of the circulatory system,both


In [29]:
dup.shape 

(417576, 35)

# HOW RO REMOVE DUPLICATES-create a unique field - row number

In [30]:
Health_State_2023.head(2)

,HS_unique_member_id_key,HS_claim_location_country_nm,HS_region,HS_icd_10_cm_code,HS_local_currency_net_amt,HS_member_id
0,11598608,United States of America (the),Americas,I2694,54.98,2.101605e+10
1,11598608,United States of America (the),Americas,I2694,84.28,2.101605e+10


In [31]:
Health_State_2023.shape

(572373, 6)

In [32]:
Health_State_2023=Health_State_2023.reset_index(drop=True)

In [33]:
Health_State_2023['Row_number']=Health_State_2023.index+1 

In [34]:
Health_State_2023.head(2) 

,HS_unique_member_id_key,HS_claim_location_country_nm,HS_region,HS_icd_10_cm_code,HS_local_currency_net_amt,HS_member_id,Row_number
0,11598608,United States of America (the),Americas,I2694,54.98,2.101605e+10,1
1,11598608,United States of America (the),Americas,I2694,84.28,2.101605e+10,2


In [35]:
Health_State_2023_map_CCSR_CCS=pd.merge(left=Health_State_2023,right=CCSR_CCS, how='left',\
                        left_on='HS_icd_10_cm_code',right_on='ICD_code_2023', indicator='ICD_CODE_BUCKET_HS_CCSR_2023') 

In [36]:
Health_State_2023_map_CCSR_CCS.shape

(572373, 27)

In [37]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER= pd.merge (left=Health_State_2023_map_CCSR_CCS, right=ICD_9_CHAPTER_NAME, how='left',\
                               left_on='CCS_LVL_1_code_2019', right_on='CCS_chapter_number_2019',\
                                indicator='CCS_LEVEL1_CHAPTER_NAME_BUCKET')

In [38]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER.shape

(572373, 31)

In [39]:
ICD_CHAPTER_2023=ICD_CHAPTER_2023.rename(columns={'CCSR_Body_System_Abbreviation_2023':'CCSR_Body_sys_ABR_mapped_HS_CODE',
                                                 'ICD_10_code_book_chapter_2023':'ICD_10_chapter_mapped_HS_CODE'})

In [40]:
HEALTH_STATE_MAP_CCSR_CCS_CHAPTER['HS_ICD_code_Abr']=HEALTH_STATE_MAP_CCSR_CCS_CHAPTER['HS_icd_10_cm_code'].str[0:3]

In [41]:
HEALTH_STATE_MAP_final= pd.merge (left=HEALTH_STATE_MAP_CCSR_CCS_CHAPTER, right=ICD_CHAPTER_2023, how='left',\
                               left_on='HS_ICD_code_Abr', right_on='CODE_range_first_three_characters',\
                                indicator='CCSR_CHAPTER_CODE_BUCKET')

In [42]:
HEALTH_STATE_MAP_final.shape

(572373, 36)

In [43]:
dup_2=HEALTH_STATE_MAP_final[HEALTH_STATE_MAP_final.duplicated(['Row_number'])] 

In [44]:
dup_2.head(3)

,HS_unique_member_id_key,HS_claim_location_country_nm,HS_region,HS_icd_10_cm_code,HS_local_currency_net_amt,HS_member_id,Row_number,ICD_code_2023,ICD_10_CM_CODE_DESCRIPTION_2023,Default_CCSR_CATEGORY_IP_2023,Default_CCSR_CATEGORY_DESCRIPTION_IP_2023,Default_CCSR_CATEGORY_OP_2023,Default_CCSR_CATEGORY_DESCRIPTION_OP_2023,CCS_CATEGORY_2019,CCS_CATEGORY_DESCRIPTION_2019,icd_code_2019,ICD_10_CM_CODE_DESCRIPTION_2019,CCS_LVL_1_code_2019,CCSR_CATEGORY_1_2023,CCSR_CATEGORY_1_DESCRIPTION_2023,CCS_LVL_1_Description_2019,CCS_LVL_2_code_2019,CCS_LVL_2_Description_2019,CCSR_Body_System_Abbreviation_2023,ICD_2023_code_abr,ICD_10_code_book_chapter_2023,ICD_CODE_BUCKET_HS_CCSR_2023,CCS_chapter_number_2019,CCS_Body_System_Abbreviation_2019,IVD_9_code_diagnosis chapters,CCS_LEVEL1_CHAPTER_NAME_BUCKET,HS_ICD_code_Abr,CODE_range_first_three_characters,CCSR_Body_sys_ABR_mapped_HS_CODE,ICD_10_chapter_mapped_HS_CODE,CCSR_CHAPTER_CODE_BUCKET


In [45]:
dup_2.shape

(0, 36)

In [ ]:
HEALTH_STATE_MAP_final.drop_duplicates()

In [ ]:
HEALTH_STATE_MAP_final=HEALTH_STATE_MAP_final.drop_duplicates(subset=['Row_number'],keep='first')   

In [ ]:
HEALTH_STATE_MAP_final.shape

## CLAIM TOTAL

In [ ]:
HEALTH_STATE_MAP_final['HS_local_currency_net_amt'].sum()

# Diamond_HS

In [ ]:
HEALTH_STATE_MAP_final.groupby('CCSR_Body_sys_ABR_mapped_HS_CODE')\
.agg({'HS_unique_member_id_key': 'count', 'HS_local_currency_net_amt':'sum'})

In [ ]:
HEALTH_STATE_MAP_final[('CCSR_Body_sys_ABR_mapped_HS_CODE')].isnull().sum()